In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/insurance_pre.csv')

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.20, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = {
    'n_estimators': [10, 20, 30, 100],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3 ,n_jobs=-1)
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30, 100]},
             verbose=3)

In [11]:
# Assuming 'grid' is your GridSearchCV object and has been fitted to the data
print("Best parameters:", grid.best_params_)
# Access the mean test score for the best parameters
best_index = grid.best_index_
best_r2_score = grid.cv_results_['mean_test_score'][best_index]
# Printing the best R² score
print("r2:",best_r2_score)

Best parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 20}
r2: 0.8338613321085644


In [12]:
table=pd.DataFrame.from_dict(grid.cv_results_)

In [13]:
table.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
213,0.040436,0.001392,0.004974,0.000386,20,log2,4,10,20,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.861121,0.838344,0.831338,0.852157,0.786346,0.833861,0.025931,1
175,0.184675,0.000914,0.015924,0.001689,20,sqrt,4,5,100,"{'max_depth': 20, 'max_features': 'sqrt', 'min...",0.862775,0.839987,0.823425,0.852428,0.785237,0.832770,0.027150,2
99,0.189035,0.002421,0.017738,0.004134,10,sqrt,4,2,100,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",0.859955,0.840837,0.827079,0.848522,0.786470,0.832573,0.025409,3
207,0.190869,0.006443,0.019498,0.006955,20,log2,4,2,100,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.861533,0.838932,0.828136,0.843035,0.789979,0.832323,0.023758,4
127,0.202945,0.008071,0.016150,0.000443,10,log2,2,5,100,"{'max_depth': 10, 'max_features': 'log2', 'min...",0.862191,0.837981,0.825891,0.852752,0.782782,0.832319,0.027702,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0.028607,0.002218,0.004197,0.000144,20,log2,1,2,10,"{'max_depth': 20, 'max_features': 'log2', 'min...",0.822425,0.812015,0.777838,0.817826,0.760163,0.798053,0.024593,284
252,0.026803,0.000871,0.003818,0.000103,30,log2,1,2,10,"{'max_depth': 30, 'max_features': 'log2', 'min...",0.832982,0.800933,0.786570,0.819981,0.748497,0.797793,0.029314,285
0,0.055033,0.009111,0.007205,0.001565,None,sqrt,1,2,10,"{'max_depth': None, 'max_features': 'sqrt', 'm...",0.829086,0.808489,0.763938,0.819154,0.755853,0.795304,0.029746,286
36,0.027437,0.000793,0.004093,0.000186,None,log2,1,2,10,"{'max_depth': None, 'max_features': 'log2', 'm...",0.841811,0.795022,0.775641,0.821078,0.736995,0.794109,0.036351,287


In [14]:
# Assuming 'grid' is your GridSearchCV object that has been fitted to the data
best_model = grid.best_estimator_

In [16]:
import pickle
filename="RF_G.sav"
pickle.dump(best_model,open(filename,'wb'))